In [277]:
import pandas as pd
import matplotlib as plt
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset


In [278]:
# #import data from CDC
# df = pd.read_csv("data\FluViewPhase2Data\WHO_NREVSS_Combined_prior_to_2015_16.csv")

# #split into training data and test data
# train = df
# # test = df[df["YEAR"]>2012]

# #turn data into features and output
# #features: 5 previous + one from last year for predicted
# #output: prediction for next time

# #create test data
# shift = 52
# train_X = np.empty([5,len(train)-shift],dtype=int)
# train_y = np.empty([1,len(train)-shift],dtype=int)
# train_Data = []
# for i in range(shift,len(train)):
#     train_X[:,i-shift] = np.asarray(train.iloc[[i-1,i-2,i-3,i-4,i-52]]["TOTAL"]).T
#     train_y[:,i-shift] = train.iloc[[i]]["TOTAL"]
#     train_Data.append((train_X,train_y))
# print(train_X[:,0])
# print(train_y[:,0])
 #import data from CDC
df = pd.read_csv("data\FluViewPhase2Data\WHO_NREVSS_Combined_prior_to_2015_16.csv")

#turn data into features and output
#features: 5 previous + one from last year for predicted
#output: prediction for next time

#create test data
shift = 52
X = np.empty([5,len(df)-shift],dtype=int)
Y = np.empty([1,len(df)-shift],dtype=int)
for i in range(shift,len(df)):
    X[:,i-shift] = np.asarray(df.iloc[[i-1,i-2,i-3,i-4,i-52]]["TOTAL"]).T
    Y[:,i-shift] = df.iloc[[i]]["TOTAL"]
X = torch.from_numpy(X)
Y = torch.from_numpy(Y)
print(X[:,1])
print(Y[:,1])


tensor([1407,    0,    0,    0, 1513], dtype=torch.int32)
tensor([1453], dtype=torch.int32)


In [279]:
#torch data set
class dataSetAll(Dataset):
    def __init__(self):
        #import data from CDC
        self.df = pd.read_csv("data\FluViewPhase2Data\WHO_NREVSS_Combined_prior_to_2015_16.csv")

        #turn data into features and output
        #features: 5 previous + one from last year for predicted
        #output: prediction for next time

        #create test data
        shift = 52
        self.X = np.empty([5,len(self.df)-shift],dtype=np.float32)
        self.Y = np.empty([1,len(self.df)-shift],dtype=np.float32)
        for i in range(shift,len(self.df)):
            self.X[:,i-shift] = np.asarray(self.df.iloc[[i-1,i-2,i-3,i-4,i-52]]["TOTAL"]).T
            self.Y[:,i-shift] = self.df.iloc[[i]]["TOTAL"]
        self.X = torch.from_numpy(self.X)
        self.Y = torch.from_numpy(self.Y)

    def __len__(self):
        return len(self.df)-52
    
    def __getitem__(self,idx):
        return self.X[:,idx], self.Y[:,idx]

In [280]:
#create data loaders
dataset = dataSetAll()
training_data, test_data = torch.utils.data.random_split(dataset, [.7,.3])
train_dataloader = DataLoader(training_data, batch_size=10)
test_dataloader = DataLoader(test_data, batch_size=10)

In [281]:
# create neural netowrk
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 3),
            nn.ReLU(),
            nn.Linear(3, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [282]:
#train/test loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % size == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [283]:
learning_rate = .003
batch_size = 10
epochs = 5
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 15187645.000000  [    0/  619]
tensor([[-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691]])
tensor([[-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691]])
tensor([[-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691]])
tensor([[-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691],
        [-3487.0691]])
tensor([[-3487.0691],
        [-3487.06